In [ ]:
"""
finds onsets using loudness
"""

import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from scipy.signal import savgol_filter
from scipy.signal import find_peaks

#opens new window
%matplotlib tk

#constants
Chunk = 2048                  #samples per frame
Format = pyaudio.paInt16        #bytes per sample
Channels = 1                    #one channel for mic
Rate = 44100                    #samples per second

#pyaudio class isntance
p = pyaudio.PyAudio()

#stream objects to get data from the microphone
stream = p.open(
    format = Format,
    channels = Channels,
    rate = Rate,
    input = True,
    output = True,
    frames_per_buffer = Chunk
)

#create matplot figure
fig, ax = plt.subplots()

#variable for plotting
x = np.arange(0, 100*Chunk, Chunk)

#create a line object - it doesn't matter what the values are rn
line, = ax.plot(x, np.random.rand(100))

#makes an array 10x the size of initial input so we can give the scrolling illusion
cumArray = np.zeros(100)

#basic formatting for the plot
ax.set_title('ONSETS')
ax.set_xlabel('sample')
ax.set_ylabel('volume (dB)')
ax.set_ylim(-20,33)
ax.set_xlim(0,100*Chunk)

k=0
comparator = 0
poop=0
run = True
onsets = []
beats = []
beatsCount = []

def onclick(event):
    global run 
    run = False

cid = fig.canvas.mpl_connect('button_press_event', onclick) #stops when we click

START = time.time()
                
while run:
    start_time = time.time()
    
    data = stream.read(Chunk, exception_on_overflow=False)  
    data_int = np.frombuffer(data, dtype = np.int16)
    
    #appends max amplitude to cumulative array
    amp = max(data_int)
    
    if amp > 20 and comparator == 0:
        comparator = amp
        
    if amp == 0 and comparator > 0:
        amp = 1
        
    dB = 10 * math.log(amp/comparator, 10)
    
    cumArray = cumArray[1:]
    cumArray = np.concatenate((cumArray, dB), axis=None)
    
    peaks = find_peaks(cumArray, height = 3, prominence = 3)
    peaks = peaks[0]        #we only take the actual array part (find_peaks() returns dict too)
    
    for i in range(len(peaks) - 1):
        delta = peaks[i+1] - peaks[i]
        unit = ellapsed_time*delta/60 #can't just use Chunk/Rate/60 because of inconsistency (1 iteration not always = 1 Chunks worth of time)
        bpm = 1 / (unit)            #converts from index in cumArray to time (in seconds)
        there = False
        for beat in beats:
            if bpm+5>beat>bpm-5:
                beatThere = beat
                there = True
        if there:
            ind = beats.index(beatThere)
            beatsCount[ind] += 1
        else:
            beats.append(int(bpm))
            beatsCount.append(1)
        
    
    #go through peaks, find time differences, if time is x larger than what's in array add it 
    #can use counter, whatever has most should be quarter
    #or use cost function, whatever produces best is full, (subtract using -log(interval/120)^2 because 120 is average bpm)
    
    for i in range(len(peaks)):
        peak = peaks[i]
        onsets.append(ax.axvline(x=peak*Chunk, color='r', linewidth=0.7))
    
    line.set_ydata(cumArray)
    fig.canvas.draw()
    fig.canvas.flush_events()
            
    #refreshes lines
    for i in range(len(onsets)):  
        onsets[i].remove()
    onsets=[]
    
    
    end_time = time.time()
    ellapsed_time = end_time - start_time
        
    k+=1
STOP = time.time()
LENGTH = STOP - START
objective = [0]*len(beats)
for i in range(len(beats)):
#     print(beats[i],  "---", beatsCount[i]/100, LENGTH/10 * math.log(beats[i]/120, 10)**2)
    objective[i] = beatsCount[i]/100 - LENGTH/10 * math.log(beats[i]/120, 10)**2
    
print(beats[objective.index(max(objective))])




/Users/srikarnimmagadda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide
